In [2]:
import requests
from utils import check_api_call_limit
from dotenv import load_dotenv
import os 
import time
load_dotenv()
# Load the api key from dotenv:
GOOGLE_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
if not GOOGLE_API_KEY:
    raise ValueError("No API key found. Please set the GOOGLE_MAPS_API_KEY environment variable in the .env file.")


In [15]:
# Lets try using the google places nearby search 
# In order to do that we need to have a lat and lng
# To get lat and lng we can use a few different sources: 
# I'm just going to use the google geocode api for now

import googlemaps
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
wellesly = gmaps.geocode(f"{'02481'}, USA")




In [22]:
import json
#print(json.dumps(wellesly, indent=4))
print(wellesly[0]['geometry']['location'])

{'lat': 42.3093812, 'lng': -71.28148159999999}


In [21]:
import googlemaps
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
wellesly2 = gmaps.geocode(f"{'02482'}, USA")
print(wellesly2[0]['geometry']['location'])

{'lat': 42.2904637, 'lng': -71.2963297}


In [31]:
import googlemaps
import requests
import time
from utils import check_api_call_limit

def get_car_washes_by_zip(api_key:str, zip_codes: str | list[str], zipcode_radius: int = 5000):
    """
    Fetch car washes for one or more zip codes using Google Places API Nearby Search.
    Includes deduplication based on place_id.

    Args:
        api_key (str): Google Maps API key.
        zip_codes (str or list): Single zip code or list of zip codes.
        zipcode_radius (int): The radius of the search in meters. Defaults to 5000.

    Returns:
        list: Car wash info (name, address, rating, etc.) if successful.
        dict: Error details if API limit exceeded.
    """
    if isinstance(zip_codes, str):  # If the zip_codes is a string, convert it to a list
        zip_codes = [zip_codes]

    gmaps = googlemaps.Client(key=api_key)
    places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    all_car_washes = {}  # Use a dictionary to store unique car washes (for deduplication)

    for zip_code in zip_codes:
        print(f"Getting coordinates for car washes in {zip_code}", end="...")
        try:
            # Fetch the coordinates for the zip code from the google geocode api
            geocode_result = gmaps.geocode(f"{zip_code}, USA")
            if not geocode_result:
                print(f"Could not find coordinates for zip code {zip_code}")
                continue
            
            # Extract the latitude and longitude from the geocode result
            location = geocode_result[0]['geometry']['location']
        except Exception as e:
            print(f"Error geocoding zip code {zip_code}: {str(e)}")
            continue
        print("done.")

        # Define the parameters for the nearby search
        params = {
            "key": api_key,
            "location": f"{location['lat']},{location['lng']}",
            "radius": zipcode_radius,  # The radius of the search in meters
            "type": "car_wash",
            "keyword": "car wash"
        }
        
        next_page_token = None
        callcount = 0

        while True:
            # Check API call limit before making the next request
            success, message, counts = check_api_call_limit("nearby_search", daily_limit=800, monthly_limit=3800)
            if not success:
                return {"error": f"Limit exceeded: {message}. Total calls: {counts['total_calls']}, Monthly calls: {counts['monthly_calls']}, Daily calls: {counts['daily_calls']}."}

            if next_page_token:
                params["pagetoken"] = next_page_token
            
            print(f"Searching for car washes with 5km radius around {location}", end="...")
            response = requests.get(places_url, params=params)
            results = response.json()
            
            if results.get("status") != "OK":
                print(f"Error for zip code {zip_code}: {results.get('status')}")
                break
            
            for place in results.get("results", []):
                place_id = place['place_id']
                if place_id not in all_car_washes:
                    all_car_washes[place_id] = {
                        "name": place["name"],
                        "address": place.get("vicinity"),
                        "goog_rating": place.get("rating"),
                        "lat": place["geometry"]["location"]["lat"],
                        "lng": place["geometry"]["location"]["lng"],
                        "goog_places_id": place_id,
                        "zip_codes": [zip_code]  # Start a list of associated zip codes
                    }
                else:
                    # If the car wash already exists, just add the current zip code to its list
                    if zip_code not in all_car_washes[place_id]["zip_codes"]:
                        all_car_washes[place_id]["zip_codes"].append(zip_code)
            
            next_page_token = results.get("next_page_token")
            callcount += 1
            if not next_page_token or callcount >= 3:  # Limit to 3 pages per zip code
                print("done.")
                break
                
            # Wait before making the next request (to comply with API usage limits)
            time.sleep(2)

    print(f"Number of zip codes searched: {len(zip_codes)}")
    print(f"Number of car washes found: {len(list(all_car_washes.values()))}")
    # Convert the dictionary of unique car washes back to a list
    return {'results': list(all_car_washes.values()), 'num_results': len(list(all_car_washes.values())), 'num_zip_codes': len(zip_codes)}

In [32]:
get_car_washes_by_zip(GOOGLE_API_KEY, ['2301', '2148', '2155', '2360', '2169',])



Getting coordinates for car washes in 2301...done.
Searching for car washes with 5km radius around {'lat': 42.0725522, 'lng': -71.0378909}...done.
Getting coordinates for car washes in 2148...done.
Searching for car washes with 5km radius around {'lat': 42.4326041, 'lng': -71.0557196}...done.
Getting coordinates for car washes in 2155...done.
Searching for car washes with 5km radius around {'lat': 42.4274971, 'lng': -71.10920120000002}...done.
Getting coordinates for car washes in 2360...done.
Searching for car washes with 5km radius around {'lat': 41.8804637, 'lng': -70.645473}...Error for zip code 2360: ZERO_RESULTS
Getting coordinates for car washes in 2169...done.
Searching for car washes with 5km radius around {'lat': 42.25136819999999, 'lng': -70.9962875}...done.
Number of zip codes searched: 5
Number of car washes found: 50


{'results': [{'name': 'Bubble Bay Car Wash',
   'address': '1015 Main St, Brockton',
   'goog_rating': 4.7,
   'lat': 42.0652871,
   'lng': -71.0164648,
   'goog_places_id': 'ChIJJ5oDGRWF5IkR5HQ8nKQsQhA',
   'zip_codes': ['2301']},
  {'name': 'Fresh Auto Wash',
   'address': '105 Depot St, South Easton',
   'goog_rating': 4,
   'lat': 42.042648,
   'lng': -71.0769825,
   'goog_places_id': 'ChIJi2aX5FyP5IkRX7lggkAfA3g',
   'zip_codes': ['2301']},
  {'name': 'Prestige Car Wash (Brockton)',
   'address': '245 N Pearl St, Brockton',
   'goog_rating': 4.5,
   'lat': 42.098091,
   'lng': -71.06555,
   'goog_places_id': 'ChIJWx_ztkWE5IkR9pEyAePesUE',
   'zip_codes': ['2301']},
  {'name': "Sunny's Car Wash",
   'address': '547 Westgate Dr, Brockton',
   'goog_rating': 3.5,
   'lat': 42.0918451,
   'lng': -71.0532332,
   'goog_places_id': 'ChIJCfJl_laE5IkRE9V8I0uQcac',
   'zip_codes': ['2301']},
  {'name': "Sunny's Car Wash",
   'address': '1075 N Montello St, Brockton',
   'goog_rating': 3.7,


In [35]:
import asyncio
from fastapi.responses import StreamingResponse

# Lets try to use a generator so we can stream some progress statements to the frontend
async def get_car_washes_by_zip(api_key: str, zip_codes: str | list[str], zipcode_radius: int = 5000):
    """
    Fetch car washes for one or more zip codes using Google Places API Nearby Search.
    Includes deduplication based on place_id and streams updates.

    Args:
        api_key (str): Google Maps API key.
        zip_codes (str or list): Single zip code or list of zip codes.
        zipcode_radius (int): The radius of the search in meters. Defaults to 5000.

    Returns:
        StreamingResponse: Streams updates and final results.
    """
    async def generate():
        if isinstance(zip_codes, str):
            zip_codes_list = [zip_codes]
        else:
            zip_codes_list = zip_codes

        yield json.dumps({"message": f"Starting search for {len(zip_codes_list)} zip codes..."}) + "\n"

        gmaps = googlemaps.Client(key=api_key)
        places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        all_car_washes = {}  # Use a dictionary to store unique car washes (for deduplication)

        for zip_code in zip_codes_list:
            yield json.dumps({"message": f"Getting coordinates for car washes in {zip_code}"}) + "\n"
            try:
                geocode_result = gmaps.geocode(f"{zip_code}, USA")
                if not geocode_result:
                    yield json.dumps({"message": f"Could not find coordinates for zip code {zip_code}"}) + "\n"
                    continue
                
                location = geocode_result[0]['geometry']['location']
            except Exception as e:
                yield json.dumps({"message": f"Error geocoding zip code {zip_code}: {str(e)}"}) + "\n"
                continue

            yield json.dumps({"message": f"Searching for car washes within {zipcode_radius}m radius of {zip_code}"}) + "\n"

            params = {
                "key": api_key,
                "location": f"{location['lat']},{location['lng']}",
                "radius": zipcode_radius,
                "type": "car_wash",
                "keyword": "car wash"
            }
            
            next_page_token = None
            callcount = 0

            while True:
                success, message, counts = check_api_call_limit("nearby_search", daily_limit=800, monthly_limit=3800)
                if not success:
                    yield json.dumps({"error": f"Limit exceeded: {message}. Total calls: {counts['total_calls']}, Monthly calls: {counts['monthly_calls']}, Daily calls: {counts['daily_calls']}."}) + "\n"
                    return

                if next_page_token:
                    params["pagetoken"] = next_page_token
                
                response = requests.get(places_url, params=params)
                results = response.json()
                
                if results.get("status") != "OK":
                    yield json.dumps({"message": f"Error for zip code {zip_code}: {results.get('status')}"}) + "\n"
                    break
                
                for place in results.get("results", []):
                    place_id = place['place_id']
                    if place_id not in all_car_washes:
                        all_car_washes[place_id] = {
                            "name": place["name"],
                            "address": place.get("vicinity"),
                            "goog_rating": place.get("rating"),
                            "lat": place["geometry"]["location"]["lat"],
                            "lng": place["geometry"]["location"]["lng"],
                            "goog_places_id": place_id,
                            "zip_codes": [zip_code]
                        }
                    else:
                        if zip_code not in all_car_washes[place_id]["zip_codes"]:
                            all_car_washes[place_id]["zip_codes"].append(zip_code)
                
                next_page_token = results.get("next_page_token")
                callcount += 1
                if not next_page_token or callcount >= 3:
                    break
                
                await asyncio.sleep(2)

            yield json.dumps({"message": f"Completed search for {zip_code}"}) + "\n"

        final_results = list(all_car_washes.values())
        yield json.dumps({
            "message": "Search complete",
            "results": final_results,
            "num_results": len(final_results),
            "num_zip_codes": len(zip_codes_list)
        }) + "\n"

    return StreamingResponse(generate(), media_type="text/event-stream")